In [1]:
# NB23 — repo root & create .github/workflows
from pathlib import Path

def find_repo_root(start: Path, must_have=("app","notebooks")) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if all((cur/m).exists() for m in must_have):
            return cur
        cur = cur.parent
    if start.name.lower()=="notebooks" and all((start.parent/m).exists() for m in must_have):
        return start.parent.resolve()
    raise FileNotFoundError("Repo root not found")

ROOT = find_repo_root(Path.cwd())
print("Repo root →", ROOT)

wf_dir = ROOT/".github"/"workflows"
wf_dir.mkdir(parents=True, exist_ok=True)
print(".github/workflows →", wf_dir.exists())


Repo root → C:\.projects\stock-direction-ml\stock-direction-ml
.github/workflows → True


In [2]:
# NB23 — sanity checks
nb14 = ROOT/"notebooks"/"14_deploy_pipeline.ipynb"
req  = ROOT/"notebooks"/"requirements.txt"
print("NB14 exists?            ", nb14.exists())
print("requirements.txt exists?", req.exists())


NB14 exists?             True
requirements.txt exists? True


In [3]:
# NB23 — write .github/workflows/daily_refresh.yml (06:00 PT daily + concurrency)
lines = [
"name: Daily Refresh (run NB14)",
"",
"on:",
"  schedule:",
"    # 06:00 PT (13:00 UTC) daily",
"    - cron: '0 13 * * *'",
"  workflow_dispatch:",
"",
"permissions:",
"  contents: write",
"",
"jobs:",
"  run-daily:",
"    concurrency:",
"      group: daily-refresh",
"      cancel-in-progress: true",
"    runs-on: ubuntu-latest",
"    steps:",
"      - name: Checkout",
"        uses: actions/checkout@v4",
"",
"      - name: Set up Python",
"        uses: actions/setup-python@v5",
"        with:",
"          python-version: '3.11'",
"",
"      - name: Install deps",
"        run: |",
"          python -m pip install --upgrade pip",
"          python -m pip install jupyter nbconvert",
"          python -m pip install -r notebooks/requirements.txt",
"",
"      - name: Execute NB14",
"        run: |",
"          jupyter nbconvert --to notebook --execute notebooks/14_deploy_pipeline.ipynb \\",
"            --output notebooks/14_deploy_pipeline_ran.ipynb \\",
"            --ExecutePreprocessor.timeout=1200 \\",
"            --ExecutePreprocessor.kernel_name=python3",
"",
"      - name: Commit & push if changed",
"        run: |",
"          git config user.name 'github-actions'",
"          git config user.email 'actions@github.com'",
"          git add -A",
"          if git diff --cached --quiet; then",
"            echo 'No changes to commit.'",
"          else",
"            git commit -m 'CI: daily refresh (NB14 executed)'",
"            git push",
"          fi",
]
wf_path = ROOT/".github"/"workflows"/"daily_refresh.yml"
wf_path.write_text("\\n".join(lines) + "\\n", encoding="utf-8")
print("Wrote:", wf_path.resolve())


Wrote: C:\.projects\stock-direction-ml\stock-direction-ml\.github\workflows\daily_refresh.yml


In [4]:
# NB23 — presence check
print((ROOT/".github"/"workflows"/"daily_refresh.yml").exists())


True


In [5]:
# NB23 — inline diagnosis of NB14 with nbclient (optional)
import nbformat
from nbclient import NotebookClient
from nbclient.exceptions import CellExecutionError
from pathlib import Path

nb_path = ROOT / "notebooks" / "14_deploy_pipeline.ipynb"
print("nb_path exists?", nb_path.exists(), "→", nb_path)

nb = nbformat.read(str(nb_path), as_version=4)
client = NotebookClient(nb, timeout=1200, kernel_name="python3", allow_errors=False, record_timing=True)

try:
    client.execute()
    print("NB14 executed successfully under nbclient.")
except CellExecutionError as e:
    print("\nNB14 FAILED during execution.\n")
    print("------- TRACEBACK (last error) -------")
    print(e)


nb_path exists? True → C:\.projects\stock-direction-ml\stock-direction-ml\notebooks\14_deploy_pipeline.ipynb
NB14 executed successfully under nbclient.


In [6]:
# NB23 — local dry run via nbconvert (Windows-safe; optional)
import sys, subprocess
nb_in  = str((ROOT/"notebooks"/"14_deploy_pipeline.ipynb").resolve())
nb_out = "14_deploy_pipeline_ran.ipynb"

cmd = [
    sys.executable, "-m", "jupyter", "nbconvert",
    "--to", "notebook", "--execute", nb_in,
    "--output", nb_out,
    "--ExecutePreprocessor.timeout=1200",
    "--ExecutePreprocessor.kernel_name=python3",
]
print("Running:", " ".join(cmd))
rc = subprocess.call(cmd)
print("Return code:", rc, " (0 = success)")


Running: c:\.projects\stock-direction-ml\stock-direction-ml\.venv\Scripts\python.exe -m jupyter nbconvert --to notebook --execute C:\.projects\stock-direction-ml\stock-direction-ml\notebooks\14_deploy_pipeline.ipynb --output 14_deploy_pipeline_ran.ipynb --ExecutePreprocessor.timeout=1200 --ExecutePreprocessor.kernel_name=python3
Return code: 0  (0 = success)
